## Spark on AWS Elastic Map Reduce.
The first step to using this is to deploy an aws emr cluster using the spark option.  

Then modify the the port setting in the security profile so that port 8192 is exposed and your ssh key pair is set correctlly.  when it comes up login to to the master with 
<pre>
$ssh -i /path-to-your-keyfile/keyfile.pem  hadoop@ipaddress-of-master
</pre>
next do the following
<pre>
$sudo pip install ipython[notebook]

$ipython profile create default

$echo "c = get_config()" > /home/hadoop/.ipython/profile_default/ipython_notebook_config.py
$echo "c.NotebookApp.ip = '*'" >> /home/hadoop/.ipython/profile_default/ipython_notebook_config.py
$echo "c.NotebookApp.open_browser = False" >> /home/hadoop/.ipython/profile_default/ipython_notebook_config.py
$echo "c.NotebookApp.port = 8192" >> /home/hadoop/.ipython/profile_default/ipython_notebook_config.py
$export PYSPARK_DRIVER_PYTHON=/usr/local/bin/jupyter
$export PYSPARK_DRIVER_PYTHON_OPTS="notebook"
$export MASTER=yarn
</pre>
Next create a file called runjup.sh with one line
<pre>
PYSPARK_DRIVER_PYTHON=/usr/local/bin/jupyter /usr/lib/spark/bin/pyspark --master yarn
</pre>
(there may be some redundancy here, but this works)
finally execute
<pre>
$nohup ./runjup.sh &
</pre>
The examples here are based on Wikipedia.   if you want to play with the full wikipedia dump do
<pre>
$hdadoop fs -mkdir /user/wiki

$curl -s -L http://dumps.wikimedia.org/enwiki/20161020/enwiki-20161020-pages-articles-multistream.xml.bz2 \
| bzip2 -cd | hadoop fs -put  - /user/wiki/wikidump-en.xml
<\pre>

The first thing we will do is look at a sample of wikipedia access logs from 2008 to 2010. It is a small sample.

In [35]:
import numpy as np
import pyspark
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

In [36]:
conf = SparkConf() \
    .setAppName("S3AccessDemo") \
    .set("spark.jars", "/home/hzq/jars/hadoop-aws-3.2.2.jar,/home/hzq/jars/aws-java-sdk-bundle-1.11.375.jar") \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

The spark context should already be there.

In [37]:
# what one does on the containerized spark-jupyter is
#sc = pyspark.SparkContext('local[*]')
#here it is
sc

In [38]:
txtfile = sc.textFile("s3a://support.elasticmapreduce/bigdatademo/sample/wiki")

Lets make suser we have at least 10 partitions. 
And split the text lines into lists along blank space separators.

In [39]:
txtfile = txtfile.repartition(10)

In [40]:
def parseline(line):
    return np.array([x for x in line.split(' ')])

In [41]:
data = txtfile.map(parseline)

we are next going to look for the page references that mention famous folks and see how may hits there are.

In [42]:
def filter_fun(row, titles):
    for title in titles:
        if row[1].find(title) > -1:
            return True
    else:
        return False

In [43]:
namelist = ['Albert_Einstein','Lady_Gaga','Barack_Obama','Richard_Nixon','Steve_Jobs', 'Bill_Clinton', 'Bill_Gates', 'Michael_Jackson', 
            'Justin_Bieber','Dante_Alighieri' 'Shakespeare', 'Byron', 'Donald_Trump', 'Hillary_Clinton', 'Werner_Heisenberg',
            'Arnold_Schwarzenegger', 'Elon_Musk', 'Nicolas_Sarkozy', 'Vladimir_Putin', 'Vladimir_Lenin', 'Karl_Marx',
            'Groucho_Marx']

In [44]:
filterd = data.filter(lambda p: filter_fun(p, namelist))

In [45]:
def mapname(row, names):
    for name in names:
        if row[1].find(name) > -1:
            return name
    else:
        return 'huh?'

The RDD filtered has only the tuples that contain one of our names in namelist.

we next map that to tuples of the form (name, reference count)

We then reduce that by the key and sum the counts.

In [46]:
remaped = filterd.map(lambda row: (mapname(row, namelist), int(row[2]) )).reduceByKey(lambda v1, v2: v1+v2)

Py4JJavaError: An error occurred while calling o169.partitions.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2197)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2654)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:258)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)


In [ ]:
remaped.takeOrdered(100, key  = lambda x: -x[1])

In [ ]:
What happens if we look for just part of the name?    

In [ ]:
snamelist = ['Einstein','Gaga','Obama','Nixon','Jobs', 'Clinton', 'Gates', 'Jackson', 
            'Bieber','Dante', 'Shakespeare', 'Byron', 'Trump', 'Heisenberg',
            'Schwarzenegger', 'Musk', 'Sarkozy', 'Putin', 'Lenin', 'Marx',
            'Marx']

In [ ]:
sfilterd = data.filter(lambda p: filter_fun(p, snamelist))
sremaped = sfilterd.map(lambda row: (mapname(row, snamelist), int(row[2]) )).reduceByKey(lambda v1, v2: v1+v2)
sremaped.takeOrdered(100, key  = lambda x: -x[1])

let's see how many page references there are here.  we can delete pages with non readable titles.

In [ ]:
bdata = data.map(lambda row: (row[1], int(row[2])))

In [ ]:
cdata = bdata.filter(lambda p:  p[0].find('%') < 0)

In [ ]:
cdata.take(100)

In [ ]:
cdata.count()

### Now look at full wikipedia dump from HDFS
these are xml file with one line per file line.   
we can extract the titles of each of the listings

In [ ]:
wikidump = sc.textFile("hdfs:///user/wiki/wikidump-en.xml")

In [ ]:
wikidump.count()

In [ ]:
wikidump.getNumPartitions()

In [ ]:
def findtitle(line):
    if line.find('<title>') > -1:
        return True
    else:
        return False

In [ ]:
titles = wikidump.filter(lambda p: findtitle(p))

In [ ]:
titles.count()

In [ ]:
titles.cache()

In [ ]:
titles.take(200)

To get more from the wikipedia xml files you need to seperate the lines for each xml file and then parse the xml to something spark can process.